In [2]:
import numpy as np
import mne
import matplotlib.pyplot as plt

import os





#file = "chb01_04.edf"
#data = mne.io.read_raw_edf(file)
#raw_data = data.get_data()
#channels = []
#for channel in raw_data:
 #   channels.append(splitperiod(channel,5,256))
#spikes = []
#for i, channel in enumerate(channels):
   # spikes.append([])
  #  for segment in channel:
 #       spikes[i].append(spikeamount(segment, 0.0001,0.04,0.07,256))
#smoothedspikes = []
#for channel in spikes:
 #   smoothedspikes.append(smoothspikes(channel,7))

#alarms = []
#for channel in smoothedspikes:
    #isdumped = 0
    #for i,spikes in enumerate(channel):
     #   if spikes>maxspikes_inter*1.3:
    #        if isdumped == 0:
   #             alarms.append(i*5)
  #              isdumped = 1
 #           print("Alarm at"+str(i*5)+"seconds")
#print(alarms)

#data = [0,0,0,0,0,0,0,0,0,0,4,4,4,0,0,0,0,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,0,0,0,0,0]
#smooth_data = smoothspikes(data,4)
#print(smooth_data)

#print(spikeamount(smooth_data, 2.99,1,4,1))




In [3]:
# funktioner

#Takes period segment, voltage threshold, min spike duration in seconds, max spike duration in seconds, and the sampling frequency, returns the amount of spikes in the segment udner those conditions.
def spikeamount(segment, spike_thresh, spike_min, spike_max, samp_freq):
    sample_count = 0
    spikes = 0
    spike_min = spike_min*samp_freq
    spike_max = spike_max*samp_freq
    for sample in segment:
        if sample > spike_thresh:
            sample_count += 1
        elif sample_count > spike_min and sample_count < spike_max:
            spikes += 1
            sample_count = 0
        else:
            sample_count = 0
    return spikes

#Takes a period as list of samples, desired segment length in seconds and the sampling frequency, returns a list of segments of the period {{},{},...{}}
def splitperiod(period, segment_length, sampling_freq):
    segments = []
    current_segment = []
    sample_seglen = segment_length * sampling_freq
    for i, sample in enumerate(period):
        current_segment.append(sample)
        if i%sample_seglen == 0 and i!=0:
            segments.append(current_segment)
            current_segment = []
    return segments
#takes list of spike amounts, returns list of smoothed spike amounts (smoothed spike amount is the average of M neighbour spike amount)
def smoothspikes(spikes, M):
    smoothspikes = []
    window_min = -(M//2)
    window_max = M//2
    for spike in spikes:
        curravg = 0
        if window_min<0:
            for i in range(0,M):
                curravg+=spikes[i]
            smoothspikes.append(curravg/M)
        elif window_max>=len(spikes):
            for i in range (0,M):
                curravg+=spikes[len(spikes)-1-i]
            smoothspikes.append(curravg/M)
        else:
            for i in range(window_min, window_max+1):
                curravg+=spikes[i]
            smoothspikes.append(curravg/M)
        window_min +=1
        window_max +=1
    return smoothspikes


In [ ]:
#test
testperiod = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,3,1,1,1,1,1,1,3,3,3,3,3,3,3,3,3,3,1,1,1,1,1,1,1,1,1,1]
testspikes = [1,2,3,4,5,6,7,8,9]
print(len(testperiod))
print(splitperiod(testperiod, 3, 1))
for segment in splitperiod(testperiod, 3,1):
   # print(spikeamount(segment, 1, 1, 3, 1))
   next
print(smoothspikes(testspikes, 3))

In [5]:
# actual data

file = "data\chb01\chb01_01.edf" #interictal data
data = mne.io.read_raw_edf(file)
raw_data = data.get_data()
# you can get the metadata included in the file and a list of all channels:
info = data.info
channels = data.ch_names
print(len(channels))
print(channels[20])
print(raw_data[0])

Extracting EDF parameters from d:\Projektkurs Epilepsi\My CNN\data\chb01\chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\johan\AppData\Local\Temp\ipykernel_16632\2228075999.py:4: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(file)


23
FT9-FT10
[-1.45934066e-04  1.95360195e-07  1.95360195e-07 ... -1.15262515e-05
 -2.93040293e-06  1.93406593e-05]


In [6]:
channels = []
for channel in raw_data:
    channels.append(splitperiod(channel, 5, 256))
#segments = splitperiod(raw_data[0], 5, 256)
spikes = []
for i,channel in enumerate(channels):
    spikes.append([])
    for segment in channel:
        spikes[i].append(spikeamount(segment, 0.0001,0.04, 0.07, 256))
smoothedspikes = []

for channel in spikes:
    smoothedspikes.append(smoothspikes(channel, 7))

maxspikes_inter = 0

for channel in smoothedspikes:
    for spikes in channel:
        if spikes>maxspikes_inter:
            maxspikes_inter = spikes
print(maxspikes_inter)


1.7142857142857142


In [18]:

# assign directory
directory = 'data\chb01'

# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        raw_data = mne.io.read_raw_edf(f).get_data()
        channels = []
        for channel in raw_data:
            channels.append(splitperiod(channel,5,256))
        spikes = []
        for i, channel in enumerate(channels):
            spikes.append([])
            for segment in channel:
                spikes[i].append(spikeamount(segment, 0.0001,0.04,0.07,256))#segment = window
        smoothedspikes = []
        for channel in spikes:
            smoothedspikes.append(smoothspikes(channel,3))
        #[[],[]]
        
        ## beraknar Interictala periodens procentuella spikes###
        # tar smoothedspikes och ger en lista för smoothed spikes mapad mot en lista med hur många gånger det värdet på smoothed spikes skett
        channel_spike_amounts = []
        channel_window_amounts = []
        for channel in smoothedspikes: # skapar en lista för varje kanal, med värdet på smoothed spikes som kan möjligen hända i ett fönster se bild
            spikeamounts = []
            for val in channel:
                if not spikeamounts.exists(val):
                    spikeamounts.append(val)
            channel_spike_amounts.append(spikeamounts)
        sorted_channel_spike_amounts=[]
        for channel in channel_spike_amounts: #sorterar varje kanals lista
            sorted_channel_spike_amounts.append(channel.sort())
        for i,channel in enumerate(smoothedspikes): 
            channel_window_amounts.append([0] * len(channel_spike_amounts[i]))#skapar en array av nollor som är lika lång som dessa möjliga lista för varje kanal
            for val in channel:
                channel_window_amounts[i][sorted_channel_spike_amounts.index(val)] += 1 #fyller den med hur många windows det antalet spikes har här
        
        #calculates the array of percentage of total windows containing a particular value of smoothspike
        WindowSpikePercentage = []
        for channel in channel_window_amounts:
            i = 0
            while i < len(channel):
                WindowSpikePercentage.append(channel[i]/sum(channel))
                i+=1
        
        #Calculate derivative #####
        for channel in channel_window_amounts:
            WindowDerivative = []
            while i < len(channel):
                der = (WindowSpikePercentage[i+1]-WindowSpikePercentage[i])/(sorted_channel_spike_amounts[i+1]-sorted_channel_spike_amounts[i])
                WindowDerivative.append(der)
                
        ###########################        
                
                
        
        sumsx = []
        sumsy = []
        for channel in smoothedspikes: # tar varje kannal smoothspike = [[0.3,0.4,...][][]] dvs den består av arrayer (kanaler) med värden som är för varje window i kanalen i en array (smoothspike)
            sumx = 0 
            sumy = 0 
            for val in channel: # tar varje värde på antal smoothspike i kanal
                sumx += val #summan av alla spikar
                sumy += 1 #antalet fönster i kanalen
            sumsx.append(sumx)
            sumsy.append(sumy)
        means = []
        for i,val in enumerate(sumsx):
            means.append(val/(sumsy[i]))
      
      
        #working #####
        #sumx = 0
        #sumy = 0
        
        #calculates mean spikes for all channels
        #for i,value in enumerate(x):
        #    sumx +=value 
         #   sumy +=y[i]
        #mean = sumx/sumy
        #########################
        #calculates mean spikes for all channels
        #for i,value in enumerate(x):
        #    sumx +=value 
        #    sumy +=y[i]
        #mean = sumx/sumy
        
        print(means)
        
        
        
        alarms = []
        for j,channel in enumerate(smoothedspikes):
            alarm = []
            
            for i, spikes in enumerate(channel):
                if :
                    alarm.append(i)
            alarms.append(alarm)
        lowest_alarm = 999999
        equal = 1
        for i,channel in enumerate(alarms):
            for alarm in channel:
                equal = 1
                for j in range(0,len(alarms)):
                    if j!=i:
                        for sample in alarms[j]:
                            if sample == alarm:
                                equal +=1
                                if equal == 3:
                                    equal =1
                                    if sample*5 < lowest_alarm:
                                        lowest_alarm = sample*5
        print("Earliest alarm at " +str(lowest_alarm)+"sec in file "+f)
        

Extracting EDF parameters from d:\Projektkurs Epilepsi\My CNN\data\chb01\chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\johan\AppData\Local\Temp\ipykernel_16632\4080443055.py:11: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw_data = mne.io.read_raw_edf(f).get_data()


[0.2980992118683363, 0.1854427445526194, 0.0013908205841446453, 0.0013908205841446453, 0.2415391747797869, 0.03337969401947146, 0.005563282336578581, 0.0013908205841446453, 0.24942049142327355, 0.020862308762169688, 0.006954102920723225, 0.02503477051460361, 0.28372739916550854, 0.17292535929531716, 0.01668984700973574, 0.06212331942512754, 0.0152990264255911, 0.02086230876216969, 0.005563282336578581, 0.0584144645340751, 0.16643486323597584, 0.09318497913769133, 0.01668984700973574]
Earliest alarm at 999999sec in file data\chb01\chb01_01.edf
Extracting EDF parameters from d:\Projektkurs Epilepsi\My CNN\data\chb01\chb01_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\johan\AppData\Local\Temp\ipykernel_16632\4080443055.py:11: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw_data = mne.io.read_raw_edf(f).get_data()


[0.22299490032452504, 0.055632823365785844, 0.011126564673157159, 0.012517385257301812, 0.2512749188687999, 0.06073249884098287, 0.0027816411682892906, 0.009735744089012515, 0.22392211404728826, 0.06026889197960131, 0.012517385257301807, 0.04728789986091792, 0.17987946221604065, 0.08159480760315262, 0.026425591098748247, 0.09596662030598045, 0.11961057023643948, 0.06722299490032456, 0.00834492350486787, 0.04079740380157626, 0.09457579972183593, 0.04126101066295778, 0.026425591098748247]
Earliest alarm at 999999sec in file data\chb01\chb01_02.edf
Extracting EDF parameters from d:\Projektkurs Epilepsi\My CNN\data\chb01\chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\johan\AppData\Local\Temp\ipykernel_16632\4080443055.py:11: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw_data = mne.io.read_raw_edf(f).get_data()


[0.24895688456189174, 0.11961057023643942, 0.20769587389893385, 0.12702828001854408, 0.5428836346777921, 0.48446917014371715, 0.12656467315716272, 0.23643949930458993, 0.49095966620305853, 0.3922114047287899, 0.18080667593880373, 0.4033379694019473, 0.24895688456189166, 0.2002781641168287, 0.32406119610570266, 0.4575799721835885, 0.5771905424200262, 0.5748725081131185, 0.2313398238293929, 0.015299026425591094, 0.383866481223922, 0.05146036161335184, 0.32406119610570266]
Earliest alarm at 715sec in file data\chb01\chb01_03.edf


NotImplementedError: Only EDF files are supported, got seizures.

In [30]:
meanNoseazure = [0.2980992118683363, 0.1854427445526194, 0.0013908205841446453, 0.0013908205841446453, 0.2415391747797869, 0.03337969401947146, 0.005563282336578581, 0.0013908205841446453, 0.24942049142327355, 0.020862308762169688, 0.006954102920723225, 0.02503477051460361, 0.28372739916550854, 0.17292535929531716, 0.01668984700973574, 0.06212331942512754, 0.0152990264255911, 0.02086230876216969, 0.005563282336578581, 0.0584144645340751, 0.16643486323597584, 0.09318497913769133, 0.01668984700973574]
meanSeazure = [0.24895688456189174, 0.11961057023643942, 0.20769587389893385, 0.12702828001854408, 0.5428836346777921, 0.48446917014371715, 0.12656467315716272, 0.23643949930458993, 0.49095966620305853, 0.3922114047287899, 0.18080667593880373, 0.4033379694019473, 0.24895688456189166, 0.2002781641168287, 0.32406119610570266, 0.4575799721835885, 0.5771905424200262, 0.5748725081131185, 0.2313398238293929, 0.015299026425591094, 0.383866481223922, 0.05146036161335184, 0.32406119610570266]
meandiff = []
for i,val in enumerate(meanNoseazure):
    meandiff.append(meanSeazure[i] - val)
print(meandiff)

meandiff = [0.4914232730644458, 0.6583217431617996, 0.2063050533147892, 0.12563745943439944, 0.3013444598980052, 0.4510894761242457, 0.12100139082058413, 0.2350486787204453, 0.24153917477978498, 0.3713490959666202, 0.1738525730180805, 0.3783031988873437, 0.3477051460361688, 0.27352804821511534, 0.3073713490959669, 0.3954566527584609, 0.5618915159944351, 0.5540101993509489, 0.22577654149281431, 0.43115438108484005, 0.21743161798794614, 0.41724617524339494, 0.3073713490959669]
for i,val in enumerate(meanNoseazure):
    if val > meanSeazure[i]:
        print('warning')

[-0.04914232730644458, -0.06583217431617996, 0.2063050533147892, 0.12563745943439944, 0.3013444598980052, 0.4510894761242457, 0.12100139082058413, 0.2350486787204453, 0.24153917477978498, 0.3713490959666202, 0.1738525730180805, 0.3783031988873437, -0.03477051460361688, 0.027352804821511534, 0.3073713490959669, 0.3954566527584609, 0.5618915159944351, 0.5540101993509489, 0.22577654149281431, -0.043115438108484005, 0.21743161798794614, -0.041724617524339494, 0.3073713490959669]
warning
warning
warning
warning
warning
